# PyTorch中搭建分类网络实例

来自b站up主deep_thoughts 合集【PyTorch源码教程与前沿人工智能算法复现讲解】

P_06_PyTorch中搭建分类网络实例：
    
https://www.bilibili.com/video/BV1VF411a7oz/?spm_id_from=333.788&vd_source=18e91d849da09d846f771c89a366ed40

transform 官方文档：https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html

build model 官方文档：https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

### 以下代码来自 build model 官方文档

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### 选择用于训练的设备

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


### 定义网络

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


## 模型层

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2780,  0.3503,  0.1463, -0.3755, -0.6236,  0.4275,  0.3753,  0.2054,
          0.3088, -0.0295,  0.0679,  0.1017, -0.3025, -0.0648,  0.2247,  0.5785,
          0.0542,  0.0841, -0.0850,  0.5290],
        [-0.0885,  0.5606, -0.0655, -0.3955, -0.3262,  0.3654,  0.3318,  0.3258,
          0.2063, -0.3415, -0.1687,  0.2529, -0.1559, -0.1788,  0.3850,  0.4877,
          0.3575,  0.0779,  0.1004,  0.3086],
        [ 0.2244,  0.3643, -0.0068, -0.4674, -0.4539,  0.0373,  0.2333, -0.2000,
         -0.0285, -0.3393,  0.2245,  0.4394,  0.1486,  0.0634,  0.1618,  0.8010,
          0.2595,  0.3017, -0.2654,  0.4620]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.3503, 0.1463, 0.0000, 0.0000, 0.4275, 0.3753, 0.2054, 0.3088,
         0.0000, 0.0679, 0.1017, 0.0000, 0.0000, 0.2247, 0.5785, 0.0542, 0.0841,
         0.0000, 0.5290],
        [0.0000, 0.5606, 0.0000, 0.0000, 0.0000, 0.3654, 0.3318, 0.3258, 0.2063,
         0.0000, 0.0000, 0.2529, 0.0000, 0.0000, 0.38

### nn.Sequential

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[-0.0943,  0.0147,  0.4421, -0.5255,  0.1280, -0.2920, -0.2450,  0.0808,
          0.1453,  0.1109],
        [ 0.0651, -0.0972,  0.4696, -0.4603,  0.2360, -0.3277, -0.2418,  0.0055,
          0.1065,  0.1505],
        [ 0.0225, -0.0393,  0.3955, -0.3633,  0.1399, -0.3544, -0.3701, -0.0119,
          0.0358,  0.2405]], grad_fn=<AddmmBackward0>)


### nn.Softmax

In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0901, 0.1005, 0.1541, 0.0586, 0.1126, 0.0740, 0.0775, 0.1074, 0.1145,
         0.1107],
        [0.1040, 0.0885, 0.1559, 0.0615, 0.1234, 0.0702, 0.0766, 0.0980, 0.1084,
         0.1133],
        [0.1022, 0.0961, 0.1485, 0.0695, 0.1150, 0.0701, 0.0690, 0.0988, 0.1036,
         0.1271]], grad_fn=<SoftmaxBackward0>)


### model parameters

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0093, -0.0210,  0.0013,  ..., -0.0142, -0.0237,  0.0008],
        [-0.0307, -0.0249, -0.0324,  ..., -0.0045,  0.0103, -0.0105]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0224, -0.0081], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0426, -0.0308, -0.0271,  ...,  0.0122,  0.0140,  0.0107],
        [ 0.0157,  0.0416, -0.0033,  ..., -0.0177,  0.0206, -0.0379]],
       device='cuda:0', grad_fn=<Sl